In [1]:
import re
import math
import pickle
import random
import pandas as pd
!pip install indic_transliteration
!pip install googletrans

from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
from googletrans import Translator

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 55 kB 126 kB/s 
     |████████████████████████████████| 972 kB 3.5 MB/s 
     |████████████████████████████████| 42 kB 199 kB/s 
     |████████████████████████████████| 53 kB 132 kB/s 
     |████████████████████████████████| 65 kB 294 kB/s 
     |████████████████████████████████| 98 kB 459 kB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15737 sha256=5ab6e4973c5fecdaf7257d8c7f1bdbae4d693c63efedf36463ad01258cf85c12
  Stored in directory: /home/bhavyajeet/.cache/pip/wheels/06/a1/90/75375ea9aff253a4803b2356007f64aecb6ce2f1e46a0aa9ba
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7665 sha256=a41e7966b4c2f68b8764e37298fb69daf03e71898e23105d3cc1c69f28322c37
  Stored in directory: /home/bhavyajeet/.cache/pip/wheels/41/11/53/91172

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Data List
nums = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,100]
hindiN = ['शून्य','एक','दो','तीन','चार','पांच','छह','सात ','आठ','नौ','दस','ग्यारह','बारह','तेरह','चौदह','पन्द्रह','सोलह','सत्रह','अठारह','उन्नीस','बीस','तीस','चालीस','पचास','साठ','सत्तर','अस्सी','नब्बे','एक सौ']

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
hindiL = ['ए', 'बी', 'सी', 'डी', 'ई', 'एफ', 'जी', 'एच', 'आई', 'जे', 'के', 'एल', 'एम','एन', 'ओ', 'पी', 'क्यू', 'आर', 'एस', 'टी', 'यू','वी', 'डब्ल्यू', 'एक्स', 'वाई', 'जेड']

hindi_letters ={letters[i]:hindiL[i] for i in range(len(letters))} 
hindi_nums = {nums[i]:hindiN[i] for i in range(len(nums))}


# Transliteration is BAD

Look at how badly it transliterates

In [4]:
translator = Translator()
data = ['Tamil Nadu', 'Maharashtra', 'Trivandrum', 'Salem', 'Trichy', 'Coimbatore', 'Pondicherry']
for place in data:
    text = translator.translate(place, src='en', dest='hi')
    print(text.text, end='  ')
    print(transliterate(place, sanscript.ITRANS, sanscript.DEVANAGARI))

# So we need to process the text before conversion as it is difficult to convert directly
print(translator.translate('104,2ND AGRAHARAM,NEAR OLD BUS STAND', src='en', dest='hi').text)

तमिलनाडु  टमिल् णदु
महाराष्ट्र  ंअहरश्त्र
त्रिवेंद्रम  ट्रिवन्द्रुम्
सलेम  Sअलेम्
त्रिची  ट्रिच्य्
कोयंबटूर  Cओइम्बतोरे
पांडिचेरी  Pओन्दिचेर्र्य्
104,2ND AGRAHARAM, NEAR OLD BUS STAND


# Play with data

In [6]:
hospitals = pd.read_excel('./HospitalData.xls')
hospitals.drop(columns=['SL NO'],axis=1,inplace=True)
hospitals.head(n = 4)
col = list(hospitals.columns)
print(col)

['ZONE', 'STATE', 'CITY', 'CATEGORY', 'HOSPITAL NAME', 'Hospital ID', 'ADDRESS', 'PINCODE', 'PHONE 1']


1. Create a Dictionary which maps columns of Dataframe to available sentences we have in HINDI.

2. Take Data from each row and translate its meaning.

3. Use this translated meaning to create a paragraph(wiki Article).

# Dictionary creation

Sentences are picked from the doc

In [7]:
sentence_dict = dict()
sentence_dict['HOSPITAL NAME'] = ['अस्पताल का नाम $NAME है| ']
sentence_dict['state_city'] = ['यह अस्पताल $STATE राज्य के $CITY नगर में स्थित है। ', '$STATE राज्य के अंतर्गत आने वाले कई अस्पतालों में से यह एक है। यह अस्पताल $CITY नगर में स्थित है।']
sentence_dict['ZONE'] = ['यह अस्पताल $ZONE जोन के अंतर्गत आता है। ','यह अस्पताल $ZONE जोन में आने वाले प्रसिद्ध अस्पतालों में से एक है। ', '$ZONE जोन में आने वाले कई नामी अस्पतालों में इसकी गिनती की जाती है। ']
sentence_dict['CATEGORY'] = ['यह एक $CATEGORY का अस्पताल है। ', 'यह $CATEGORY में आने वाले कई नामी अस्पतालों में से एक है। ', 'यह अस्पताल $CATEGORY  के लिए मशहूर है। ']
sentence_dict['Hospital ID'] = ['इस अस्पताल का क्रमांक $HOSPITALID है। ']
sentence_dict['ADDRESS'] = ['इसका पता है : $ADDRESS । ', 'यह $ADDRESS में स्थित है। ']
sentence_dict['PINCODE'] = ['इसका पिन कोड है : $PINCODE । ']
sentence_dict['PHONE 1'] = ['इसका फ़ोन नंबर है : $PHONE । ', 'इस अस्पताल से संपर्क करने के लिए $PHONE फ़ोन नंबर का प्रयोग करें। ', 'इस अस्पताल की सभी सुविधाओं के बारे में जानकारी प्राप्त करने क लिए $PHONE को डायल करें। ']

should_translate = [1, 1, 1, 1, 1, 0, 1, 0, 0]

In [11]:
'''
Replace $ with the given word
'''
def completed_sentence(sentence, substitute):
    words = sentence.split()
    new_words = []
    for word in words:
        if word[0] == '$':
            new_words.append(str(substitute))
        else:
            new_words.append(str(word))
    return ' '.join(new_words)
'''
current_row is a dictionary with keys from the data frame
'''
def create_article(current_row):
    article = ""
    state = ""
    city = ""
    trans = 0
    for key in current_row.keys():
        if should_translate[trans] == 1:
            translated = translator.translate(str(current_row[key]), src='en', dest='hi')
            current_row[key] = translated.text
        trans += 1
        if key == 'STATE':
            state = current_row[key]
        elif key == 'CITY':
            city = current_row[key]
        else:
            ind = random.randint(0, len(sentence_dict[key]) - 1)
            article += completed_sentence(sentence_dict[key][ind], current_row[key])
    
    ind = random.randint(0, len(sentence_dict['state_city']) - 1)
    words = sentence_dict['state_city'][ind]
    words = words.split()
    new_words = []
    first = False
    for word in words:
        if word[0] == '$':
            if not first:
                new_words.append(state)
                first = True
            else:
                new_words.append(city)
        else:
            new_words.append(word)
    article += ' '.join(new_words)
    return article

In [8]:
print(hospitals.shape)
hospitals.info()
pd.set_option('display.max_columns', 50)

(5483, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5483 entries, 0 to 5482
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ZONE           5483 non-null   object
 1   STATE          5483 non-null   object
 2   CITY           5483 non-null   object
 3   CATEGORY       5483 non-null   object
 4   HOSPITAL NAME  5483 non-null   object
 5   Hospital ID    5483 non-null   object
 6   ADDRESS        5483 non-null   object
 7   PINCODE        5483 non-null   object
 8   PHONE 1        5480 non-null   object
dtypes: object(9)
memory usage: 385.6+ KB


In [9]:
hospitals[:][:4]

,ZONE,STATE,CITY,CATEGORY,HOSPITAL NAME,Hospital ID,ADDRESS,PINCODE,PHONE 1
0,SOUTH,TAMILNADU,SALEM,GENERAL HOSPITAL,A.C.HOSPITAL,HOS-CHE-1150,"104,2ND AGRAHARAM,NEAR OLD BUS STAND",636001,(0427)2260577/2267957
1,SOUTH,TAMILNADU,COIMBATORE,GENERAL HOSPITAL,A.G.HOSPITAL,HOS-COM-2275,"34, KPN COLONY 3RD STREET,",641601,0421-2202444
2,SOUTH,TAMILNADU,PONDICHERRY,GENERAL HOSPITAL,A.G.PADMAVATHI'S HOSPITAL LTD,HOS-CHE-470,"#12 VICTORIYA NAGAR,ELLAIPILLAI CHAVADY",605005,(0413) 2204620/2204621
3,SOUTH,KERALA,TRIVANDRUM,GENERAL HOSPITAL,A.J.HOSPITAL,HOS-KOC-2595,KAZHAKUTTOM,695582,0471-2418452


In [12]:
position = dict()
position['1st'] = 'first'
position['2nd'] = 'second'
position['3rd'] = 'third'
position['4th'] = 'fourth'
position['6th'] = 'sixth'
position['7th'] = 'seventh'
position['8th'] = 'eighth'
position['9th'] = 'ninth'
'''
process for better translation
'''
def process(data):
    data = data.split()
    new_data = []
    for d in data:
        d = str(d)
        d = d.lower()
        if d in position:
            new_data.append(position[d])
        else:
            new_data.append(d)
    return ' '.join(new_data)
            
for i in range(4):
    current_row = dict()
    for k in col:
        current_row[k] = process(str(hospitals[k][i]))
    article = create_article(current_row)
    print(article)


दक्षिण जोन में आने वाले कई नामी अस्पतालों में इसकी गिनती की जाती है।यह अस्पताल सामान्य अस्पताल के लिए मशहूर है।अस्पताल का नाम a.c.hospital है|इस अस्पताल का क्रमांक hos-che-1150 है।इसका पता है : पुराना बस स्टैंड के पास 104,2, अग्रहारम ।इसका पिन कोड है : 636001 ।इस अस्पताल की सभी सुविधाओं के बारे में जानकारी प्राप्त करने क लिए (0427)2260577/2267957 को डायल करें।यह अस्पताल तमिलनाडु राज्य के सलेम नगर में स्थित है।
दक्षिण जोन में आने वाले कई नामी अस्पतालों में इसकी गिनती की जाती है।यह सामान्य अस्पताल में आने वाले कई नामी अस्पतालों में से एक है।अस्पताल का नाम a.g.hospital है|इस अस्पताल का क्रमांक hos-com-2275 है।इसका पता है : 34, kpn कॉलोनी तीसरी गली, ।इसका पिन कोड है : 641601 ।इस अस्पताल से संपर्क करने के लिए 0421-2202444 फ़ोन नंबर का प्रयोग करें।यह अस्पताल तमिलनाडु राज्य के कोयंबटूर नगर में स्थित है।
यह अस्पताल दक्षिण जोन के अंतर्गत आता है।यह सामान्य अस्पताल में आने वाले कई नामी अस्पतालों में से एक है।अस्पताल का नाम a.g.padmavathi's अस्पताल लि है|इस अस्पताल का क्रमांक hos-che-470 है।यह # 1